In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from movies import *

In [2]:
pd.plotting.register_matplotlib_converters()
plt.rcParams['figure.figsize'] = [12, 8]
# np.warnings.simplefilter(action='ignore')
np.set_printoptions(linewidth=120, suppress=True)

In [9]:
import feedparser


## get scripts to import
scripts = []
feed_url = "https://www.imsdb.com/feeds/alphabetical.php?letter=%s"
alphabet = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
for letter in alphabet:
    res = feedparser.parse(feed_url%letter)
    res = pd.DataFrame(res['entries'])
    scripts = scripts + list(res['link'])
scripts = list(map(lambda x: x[29:-5], scripts))

In [ ]:
%%time
## get subset of scripts
use_scripts = np.random.choice(scripts, 100, replace=False)
## import scripts...
import_scripts(use_scripts, data_dir="scripts/", wait=True)

Bender-Gets-Made
Five-Easy-Pieces
The-Farnsworth-Parabox
Departed,-The
Saving-Mr.-Banks
Taking-Lives


In [ ]:
%%time
## load data
scripts = load_scripts(data_dir='scripts/', tokenize=True)
vocab, dataset = build_vocab(scripts, min_count=4)
pipeline = dataset_generator(dataset, n_batch=64, n_seq=64)

## initialize model
model = SimpleRNN(len(vocab), n_embed=64, n_hidden=256, n_layers=4, dropout=0.10)

## train model
w = corpus_invfreq(dataset, sqrt=True)
model, logs = training(model, pipeline, w=w, lr=0.1, n_epochs=16, n_steps=64)
hist = logs.groupby('epoch').mean()

In [ ]:
## plot loss
plt.plot(hist['loss'], 'o-')
plt.show()